In [1]:
import pandas as pd
import numpy as np
import copy
import networkx as nx
from datetime import date
# elo_df = pd.read_csv("Current ELO.csv")

np.set_printoptions(suppress = True)

class TournamentOrganizer():
    """
    A tournament organizer class that will attempt to do everything that the tournament will need.
    This includes saving match_history, creating pairings, providing those pairings in an easy to
    read format, etc.
    """
    def __init__(self, player_list, tournament_history = None, initial_seeding = False):
        self.player_list = player_list
        # If given, tournament history should be a dictionary containing 
        # Round: int // Player 1: str // Player 2: str // Score: str (X-X) // Result: str ("W" or "D")
        # as key-value pairs
        self.round_num = 0
        self.match_num = 0
        if tournament_history is None:
            self.tournament_history = dict()
        else:
            # Consider adding functionality that loads/saves data 
            # halfway through the tournament
            # e.g. get entry numbers, have a round_complete boolean
            self.tournament_history = tournament_history
            # In the case where loading from empty tournament history
            try:
                self.round_num = self.tournament_history[max([num for num in self.tournament_history.keys()])]["Round"]
            except:
                self.round_num = 0
            try:
                self.match_num = max([num for num in self.tournament_history.keys()])
            except:
                self.match_num = 0
        self.standings_df = None
        self.sorted_df = None
        # Whether to seed the first round based on the order of name inputs
        self.initial_seeding = initial_seeding
        self.update_standings()
        self.cost_array = None
        
    # Create self.standings_df based on self.tournament_history
    # Quite inefficient atm, but for small sizes should be okay
    def update_standings(self, sort_by_elo = False):
        """
        Creates a standings_dictionary from known player_list and tournament history.
                
        Standings can be sorted by ELO (default False)
        
        Note that we create standings from tournament history because there are cases
        where we need to undo a tournament round.
        """
        # We have an opponents key, but we'll get rid of it after calculating OMW
        standings_dict = {
            "Player": self.player_list,
        }
        
        # If the tournament history is empty, then entire dictionary should be zeroes
        if not self.tournament_history:
            for key in ["Played", "Points", "OMW", "Score For", "Score Against", "Opponents"]:
                standings_dict[key] = [0 for x in range(len(self.player_list))]
            self.standings_df = pd.DataFrame.from_dict(standings_dict)
            return
        else:
            for key in ["Played", "Points", "OMW", "Score For", "Score Against", "Opponents"]:
                standings_dict[key] = list()
                
        # For each player 
        for player in self.player_list:

            played = 0
            points = 0
            score_for = 0
            score_against = 0
            opponents = []

            # Check to see if they were involved in a match
            for match in self.tournament_history:
                # If they were player 1
                if player == self.tournament_history[match]["Player_1"]:
                    # Increased played by 1
                    played += 1
                    # Fill the opponents list with the names of each opponent
                    opponents.append(self.tournament_history[match]["Player_2"])

                    # Update the scores for and against that player
                    score_for += float(self.tournament_history[match]["Score"].split("-")[0])
                    score_against += float(self.tournament_history[match]["Score"].split("-")[1])

                    # If they won, add 1 point
                    if self.tournament_history[match]["Result"] == "W":
                        points += 1
                    # If they drew, add 0.5 point
                    elif self.tournament_history[match]["Result"] == "D":
                        points += 0.5

                # If they were player 2
                elif player == self.tournament_history[match]["Player_2"]:
                    played += 1

                    # Update the scores for and against that player
                    # Note that if they are player 2
                    score_for += float(self.tournament_history[match]["Score"].split("-")[1])
                    score_against += float(self.tournament_history[match]["Score"].split("-")[0])

                    opponents.append(self.tournament_history[match]["Player_1"])

                    # If they drew, add 0.5 point
                    if self.tournament_history[match]["Result"] == "D":
                        points += 0.5

            # Once we've looped through all the matches
            # Append the stats to the standings_dict
            standings_dict["Played"].append(played)
            standings_dict["Points"].append(points)
            standings_dict["Score For"].append(score_for)
            standings_dict["Score Against"].append(score_against)
            standings_dict["Opponents"].append(opponents)

        # We can only calculate OMW after all matches have been added to the standings
        for opponent_list in standings_dict["Opponents"]:
            running_omw = []
            # For each opponent they've previously played, find their win percentage.
            for opponent in opponent_list:
                try:
                    # Find index of their opponent
                    opponent_index = standings_dict["Player"].index(opponent)

                    running_omw.append(standings_dict["Points"][opponent_index] / 
                                       standings_dict["Played"][opponent_index])

                # If the opponent was a "bye" (i.e., they're not in the standings_dict)
                # They count as an opponent with 0% wins
                # Note that this counts players who dropped from the tournament as players
                # who have a win% of zero.
                except:
                    running_omw.append(0)
            
            # If running_omw is empty
            if not running_omw:
                # Set OMW to zero.
                standings_dict["OMW"].append(0)
            else:
                # Else, get the average and round to 3 decimal places.
                standings_dict["OMW"].append(np.round(np.mean(running_omw), 3))

        # Remove the opponents key:value pair
        standings_dict.pop("Opponents")

        # Turn the dictionary into a dataframe
        self.standings_df = pd.DataFrame.from_dict(standings_dict).sort_values(["Points", "OMW", "Score For", "Score Against"], ascending = False)
        return
    
    # Will determine pairings based on the Standings - uses Graph theory
    # Returns list of lists
    def determine_pairings(self):
        """
        Determines pairings for a given standings

        standings_df should have:
            "Player", "Points", "OMW", "ELO"

        Experimental new method of calculating pairings.
        
        Returns a list of lists
        """
        pairings = []
        
        if self.round_num == 1 and self.initial_seeding:
            self.sorted_df = self.standings_df.sort_values(["Points", "OMW"], ascending = False)
        else:
            self.sorted_df = self.standings_df.sample(frac = 1).sort_values(["Points", "OMW"], 
                                                                       ascending = False)
        
        sorted_player_list = list(self.sorted_df["Player"])
        
        
        # If there are an odd number of players, we'll need to identify a "Bye" player
        if len(sorted_player_list) % 2 != 0:
            # Iterate backwards starting from the worst player
            for player in sorted_player_list[-1::-1]:
                # We need to check if the worst performing player already had a bye. 
                # In which case they cannot have a bye again.
                if self.has_bye(player):
                    continue
                # If the player doesn't have a bye, then pop that player from the list
                # and give them a bye
                else:
                    pairings.append([sorted_player_list.pop(sorted_player_list.index(player)), "Bye"])
                    break
        
        # sorted_player_list should now be even
        
        # Create a dictionary that maps player names to their index
        name_mapping_dict = {}
        for index, name in enumerate(sorted_player_list):
            name_mapping_dict[name] = index
        
        # print(sorted_player_list)
        
        # Get the coordinates of players who have already played
        # Note that it very smartly only checks people who don't have the bye
        previously_played = self.previously_played(name_mapping_dict, sorted_player_list)
        
        # print("\nPreviously played: ", previously_played)
        # print("Sorted List: ", sorted_player_list)
        # Create a grid of "costs"
        # players = np.zeros((len(sorted_player_list), len(sorted_player_list)))
        players = np.array([i for i in range(len(sorted_player_list))])
        # print("players = ", players)
        player_array = -np.abs(players[:,None] - players)
        # print("players = \n", player_array)
        
        if previously_played:
            # Give previous pairings a HUGE cost.
            player_array[tuple(np.array(previously_played).T)] += 1000000
            # [::-1] reverses the tuple
            player_array[tuple(np.array(previously_played).T)[::-1]] += 1000000
        
        # Create a cost array for players with different scores. 
        # The cost is higher if the difference between your points is higher
        # We want it to be high, but not as high as previously_played
        scores = np.array(self.sorted_df[self.sorted_df['Player'].isin(sorted_player_list)]["Points"])
        # print("scores = ", scores)
        
        # Adjust this algorithm for different pairing variations
        score_array = (np.abs(scores[:,None] - scores)*5)**2
        # print("score_array = \n", score_array)
        
        self.cost_array = player_array + score_array
        # print("Cost Array = \n", self.cost_array)
        
        # Convert our cost array into a graph, and find the pairs that give the least cost
        # Note that min_weight_matching doesn't work - for some reason the algorithm is flawed
        # However, making the graph negative (*-1) and using max_weight_matching works.
        G = nx.Graph(self.cost_array*-1)
        pairs = nx.max_weight_matching(G, maxcardinality = True)
        
        # Return the generated pairings
        for p in list(pairs):
            result = map(lambda x: sorted_player_list[x], p)
            pairings.append(list(result))
        
        return pairings
    
    # Add result of match to tournament_history
    def add_result(self, 
                   player_one, 
                   player_two, 
                   player_one_score, 
                   player_two_score):
        """
        Prompts user for result and records it in the given dictionary

        Arguments:

        match_num

        player_one - string
        player_two - string
        player_one_score
        player_two_score

        tournament_history - dictionary with the match numbers as keys to a dictionary
            "Round"
            "Player_1" 
            "Player_2"
            "Score"
            "Result"

        Does not return anything, merely updates the tournament_history dictionary with the
        results
        """
        self.match_num += 1
        # Note that the "Result" is from the perspective of Player_1 - it should either be
        # W or D
        
        # If player one won
        if player_one_score - player_two_score > 0.0001:
            self.tournament_history[self.match_num] = {
                'Round': self.round_num,
                'Player_1': player_one,
                'Player_2': player_two,
                'Score': str(player_one_score) + "-" + str(player_two_score),
                'Result': "W"
            }
        # If player two won
        elif player_two_score - player_one_score > 0.0001:
            self.tournament_history[self.match_num] = {
                'Round': self.round_num,
                'Player_1': player_two,
                'Player_2': player_one,
                'Score': str(player_two_score) + "-" + str(player_one_score),
                'Result': "W"
            }

        # If draw
        else:
            self.tournament_history[self.match_num] = {
                'Round': self.round_num,
                'Player_1': player_one,
                'Player_2': player_two,
                'Score': str(player_one_score) + "-" + str(player_two_score),
                'Result': "D"
            }
    
    # Check if player has a bye, returns Boolean
    def has_bye(self, player):
        """
        Looks through match_history to see if the player has had a bye

        returns Boolean
        """
        if player not in self.player_list:
            print(f"Error: {player} is not in the tournament.")
            return
        
        for match in self.tournament_history:
            if player == self.tournament_history[match]["Player_1"]:
                if self.tournament_history[match]["Player_2"] == "Bye":
                    return True
            # Note that this should pretty much never happen, but include it just in case.
            elif player == self.tournament_history[match]["Player_2"]:
                if self.tournament_history[match]["Player_1"] == "Bye":
                    return True

        return False

    # Previously played
    def previously_played(self, name_mapping_dict, player_list = None):
        """
        Returns a history of all players who have already played in the form
        (Player1, Player2) based on a mapping dictionary
        """
        previously_played = []
        
        if player_list == None:
            player_list = self.player_list
        
        for match in self.tournament_history:
            if self.tournament_history[match]["Player_1"] in player_list and \
                self.tournament_history[match]["Player_2"] in player_list:
                previously_played.append((name_mapping_dict[self.tournament_history[match]["Player_1"]],
                                         name_mapping_dict[self.tournament_history[match]["Player_2"]]))

        return previously_played
    
    def undo_results(self):
        """
        Undo previous results button.
        
        Deletes all entries in tournament_history from the latest round.
        """
        # Finds all items where Round == previous round
        self.tournament_history = {key:val for key, val in self.tournament_history.items() if val["Round"] != self.round_num}
        # Reduce round number
        self.round_num -= 1
        # Reduce match number
        self.match_num = max([num for num in test_dict.keys()])
        
    

In [2]:
import tkinter as tk
from tkinter import ttk # Submodule for additional widgets
from tkinter.messagebox import showinfo
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename

class OptionsFrame(tk.Frame):
    def __init__(self, root, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        
        self.master = root
        
    
# The frame that allows you to load from csv.    
class RetrieveFrame(tk.Frame):
    # This allows the class to take all the Frame keyword args
    def __init__(self, root, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        
        self.master = root
        # Create a label
        self.label = tk.Label(self, text = "You can choose to retrieve player\n"
                          "names from a .csv file.\n\nAll player names should be in a\n"
                          "column called 'Player'.")
        # Note that you need to pack the label within the Frame itself.
        self.label.grid(row = 0, column = 1, sticky = "nsew")
        
        self.button = tk.Button(self, text = "Browse", command = self.players_from_csv)
        self.button.grid(row = 0, column = 2, padx = 15, pady = 50)
        
        self.separator = ttk.Separator(self, orient='horizontal')
        self.separator.grid(row = 1, column = 1, columnspan = 2, sticky = "ew")
        
        self.continue_label = tk.Label(self, text = "Continue a previous\ntournament.\n\n"
                          "Recommend only using\nmatch histories created\nby this program.")
        self.continue_button = tk.Button(self, text = "Browse", command = self.master.get_match_history)
        self.continue_label.grid(row = 2, column = 1, sticky = "nsew")
        self.continue_button.grid(row = 2, column = 2, padx = 15, pady = 50)
        
    def players_from_csv(self):
        """
        Loads Player Names from csv.
        
        NB: Current implementation Will not load tournament results - may choose to do this
        in a later version
        """
        filepath = askopenfilename(
            filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
        )
        
        # If no file selected, do nothing
        if not filepath:
            return
        
        # Read the csv
        temp_df = pd.read_csv(filepath)
        
        # Need to do error handling for this step
        try:
            for player in temp_df["Player"]:
                self.master.inputframe.add_player(player)

            # Get rid of any error messages if the file is correctly produced
            # Specifically, check if the error_frame exists, and if it does, destroy it.
            self.master.close_error()

        except:
            # Create an error message if there is an error.
            self.master.raise_error("Error: The file did not\ncontain a list of players.\n\n"
                                    "Check that the column is\ncorrectly labelled 'Player'")
            return
        
class ErrorFrame(tk.Frame):
    def __init__(self, root, error_message = None, *args, **kwargs):
        # Remmeber that you need to pass all the stuff INTO the super() function
        super().__init__(root, *args, **kwargs)
        
        self.error_message = tk.Label(self, 
                                     text = error_message, 
                                     fg = "red")
        self.error_message.pack()
        
        
# The frame that allows to you create multiple Player bars
class InputFrame(tk.Frame):
    def __init__(self, root, *args, **kwargs):
        # Remmeber that you need to pass all the stuff INTO the super() function
        super().__init__(root, *args, **kwargs)
        self.master = root
        
        # Create a num_player attribute - we need this to increase the row
        # Its a bit janky but it will do
        self.num_player = 0
        
        # self.add_player_button = tk.Button(self,  text = "Add Player", command = self.add_player)
        # self.add_player_button.grid(row = self.num_player + 2, column = 0, pady = 5)
        
        
    def add_player(self, player_name = None):
        # A function of the InputFrame
        # Will add a new Playerframe into the Inputframe
        self.num_player += 1
        self.playerframe = PlayerFrame(self, player_name, borderwidth=2, relief="groove")
        self.playerframe.grid(row = self.num_player, column = 0)
        
class PlayerFrame(tk.Frame):
    # Create a player frame containing a label, entry, and remove player button
    # Give the option to add a player_name as an argument
    def __init__(self, root, player_name = None, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        
        self.label = tk.Label(self, text = f"Player Name")
        
        # Create an entry for the player name
        self.entry = tk.Entry(self)
        # If a player_name was provided, fill in the field
        if player_name is not None:
            # Note that insert has the format insert(start_point, text)
            self.entry.insert(0, player_name)
        
        self.button = tk.Button(self, text = "Remove Player", command = self.destroy)
        
        self.label.grid(row = 0, column = 0)
        self.entry.grid(row = 0, column = 1)
        self.button.grid(row = 0, column = 2)
        
class SetupApplication(tk.Tk):
    def __init__(self):
        super().__init__()
        
        # configure the root window
        self.title('Player List')
        self.rowconfigure(0, minsize = 10, weight = 1)
        self.columnconfigure([0,1], minsize = 150, weight = 1)
        
        # Embed the retrievecsv Frame into the Main Window
        self.retrieveframe = RetrieveFrame(self, relief = tk.RAISED, borderwidth = 2)
        self.retrieveframe.grid(row = 1, column = 1, sticky = "nsew")
        
        self.inputframe = InputFrame(self, relief = tk.GROOVE, borderwidth = 2)
        self.inputframe.grid(row = 1, column = 0)
        
        # Create an Add Player Button
        self.add_player_button = tk.Button(self,
                                           text = "Add Player", 
                                           # The button controls the add player functionality
                                           command = self.inputframe.add_player)
        self.add_player_button.grid(row = 2, column = 0, pady = 5)
        
        # Create a Begin Tournament Button
        self.begin_tourney_button = tk.Button(self,
                                              text = "Begin Tournament", 
                                              # The button controls the add player functionality
                                              command = self.setup_organizer)
        self.begin_tourney_button.grid(row = 3, column = 0, pady = 5)
        
        
        # For winfo_exists() to work, you need to have at least created the errorframe before.
        self.errorframe = ErrorFrame(self)
        
        # Set some defaults
        self.tournamentorganizer = None
        self.tournament_history = None
        self.previous_pairings = None
        self.previous_scores = None
        
    def raise_error(self, error_message):
        self.errorframe = ErrorFrame(self, error_message)
        self.errorframe.grid(row = 4, column = 0, pady = 5)
        
    def close_error(self):
        if self.errorframe.winfo_exists():
            self.errorframe.destroy()
    
    # Create a tournament from a given CSV file.
    def get_match_history(self):
        filepath = askopenfilename(
            filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
        )

        # If no file selected, do nothing
        if not filepath:
            return
        
        self.previous_pairings = []
        self.previous_scores = []
        
        # Check add previous pairings to the previous_pairings 
        with open(filepath, "r") as f:
            for line in f.readlines():
                if 'pairing' in line:
                    self.previous_pairings.append(line.strip("\n").split(",")[1:3])
                    self.previous_scores.append(line.strip("\n").split(",")[3:])
                if 'player_list' in line:
                    # Extract the list of players in the appropriate line
                    player_list = [player.strip("'") for player in line[line.find("[")+1:line.find("]")].split(", ")]
                
        temp_df = pd.read_csv(filepath, index_col = 0, comment='#')
        
        
        # Get the player_list, which is the union of all player names
        # Note that this assumes that there are at least one round of results - let's fix this.
        # player_list = list(set(temp_df["Player_1"]).union(set(temp_df["Player_2"])))
        
        # Remove Byes if any - they're not a player!
        #if "Bye" in player_list:
        #    player_list.pop(player_list.index("Bye"))
        
        # Create the tournament history dictionary which will be passed to Main Application
        tournament_history = temp_df.transpose().to_dict()
        
        # Add all players to the player list - you can drop players pre-emptively
        # Maybe remove this functionality if loading tournament - just drop manually from 
        # the next screen
        for player in player_list:
            self.inputframe.add_player(player)
        
        self.tournament_history = tournament_history
        self.close_error()
        # self.raise_error("Match history loaded.\nYou may choose to drop players before the tournament starts.\nClick 'Begin Tournament' to start.")
        self.raise_error("Match history loaded.\nPlease do not drop players before the tournament:\nPrevious pairings have already been saved")
        
        # If there was no pairing information, set the pairings and scores to None
        if not bool(self.previous_pairings):
            self.previous_pairings = None
            self.previous_scores = None
        return
    
    def setup_organizer(self):
        """
        Sets up an instance of the Tournament Organizer class using the
        information that we have
        """
        self.close_error()
        
        # returns the list of all frames in self.inputframe
        list_of_playerframes = [i for i in self.inputframe.winfo_children() if i.winfo_class() == "Frame"]
        
        # No need to destroy the errorframe on this one since 
        if len(list_of_playerframes) < 3:
            self.raise_error("Error: A Swiss Tournament cannot be run\nwith less than three players.\nPlease add another player.")
            return
        
        player_list = list()
    
        # For each frame, retrieve the entry containing the name of the player
        # With this, no need to keep track of entry instances in list
        for frame in list_of_playerframes:
            # Each frame has multiple children - Label, Entry, Button
            for child in frame.winfo_children():
                # We want the entry - get the contents and append that to a list
                if child.winfo_class() == "Entry":
                    player_list.append(child.get())
        
        # Edit this later once I update OptionFrames
        initial_seeding = False
        self.tournamentorganizer = TournamentOrganizer(player_list = player_list, 
                                                       tournament_history = self.tournament_history)
        
        self.destroy()
        
        # We can setup the Main Application from within the old application
        app = MainApplication(self.tournamentorganizer, 
                              previous_pairings = self.previous_pairings, 
                              previous_scores = self.previous_scores)
        



In [3]:
# This frame contains all the buttons and options available to the user
class ButtonFrame(tk.Frame):
    def __init__(self, root, pairings, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        self.master = root
        # NOTE - numrounds should come from tournamentorganizer
        self.num_rounds = 1
        
        # This buffers the BOTTOM of the screen. Very nice. <- check if this works
        self.rowconfigure(6, weight = 1)
        self.columnconfigure(1, weight = 1)
        
        # Submit Results button
        self.submit = tk.Button(self, text = "Submit Results", 
                                # self.master.master = function in main window
                                command = self.master.master.submit_results)
        self.submit.grid(row = 1, column = 1, columnspan = 2, padx = (100,0), sticky = "ew")
        
        # NOTE - numrounds should come from tournamentorganizer - edit this later
        if self.master.master.tournamentorganizer.round_num < 1:
            self.undo_button = tk.Button(self, 
                                         text = "Undo Previous Round", 
                                         state = tk.DISABLED)
        else:
            self.undo_button = tk.Button(self, 
                                         text = "Undo Previous Round", 
                                         command = self.master.master.undo_results)
        self.undo_button.grid(row = 2, column = 1, columnspan = 2, padx = (100,0), sticky = "ew")
        
        self.save_history = tk.Button(self, 
                                     text = "Save Current Match History",
                                     command = self.master.master.save_history)
        self.save_history.grid(row = 3, column = 1, columnspan = 2, padx = (100,0), sticky = "ew")
        
        # Possibly remove this and make it mandatory to save pairings
#         self.round_incomplete = tk.IntVar()
#         self.round_incomplete_check = tk.Checkbutton(self, 
#                                         text = "Save Incomplete Round",
#                                         variable = self.round_incomplete)
#         self.round_incomplete_check.grid(row = 3, column = 2, padx = 30, sticky = "ew")
        
        # Create widgets for dropping players from the tournament.
        self.drop_entry = tk.Entry(self)
        self.drop_button = tk.Button(self, 
                                     text = "Drop Player (One name per click)", 
                                     command = self.master.master.drop_player)
        self.drop_entry.grid(row = 4, column = 1, pady = 5, padx = (100,0), sticky = "ew")
        self.drop_button.grid(row = 4, column = 2, pady = 5, padx = (20,0), sticky = "ew")

        # This provides users with the option to end the tournament immediately.
        # end_tourney will be 1 if the box is ticked.
        self.end_tourney = tk.IntVar()
        self.end_tournament = tk.Checkbutton(self, 
                                        text = "End the Tournament upon submission of results",
                                        variable = self.end_tourney)
        self.end_tournament.grid(row = 5, column = 1, columnspan = 2, pady = 5, padx = (100,0), sticky = "ew")


class ResultFrame(tk.Frame):
    def __init__(self, root, pair, is_bye = False, score = None, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        self.master = root
        self.pair = pair
        self.score = score
        
        self.columnconfigure([1,5], minsize = 100)
        
        self.is_bye = is_bye
        
        if self.is_bye:
            self.label = tk.Label(self, text = f"{self.pair[0]} has a bye.")
            self.label.grid(row = 1, column = 1, columnspan = 5, padx = 215, sticky = "nsew")
        
        else:
            self.label_1 = tk.Label(self, text = f"{self.pair[0]}")

            # Create a spinbox for the scoring
            # We can use .get() to get the value of the spinbox
            # If there are previous scores, insert them
            
            self.score_1 = ttk.Spinbox(self, from_ = 0, to = 100)
            self.score_2 = ttk.Spinbox(self, from_ = 0, to = 100)
            
            if self.score is not None:
                self.score_1.set(score[0])
                self.score_2.set(score[1])
            
            self.label_2 = tk.Label(self, text = f"{self.pair[1]}")
            
            self.label_1.grid(row = 1, column = 1, columnspan = 2, sticky = "e")
            self.score_1.grid(row = 1, column = 3, padx = 15, pady = 5)
            self.score_2.grid(row = 1, column = 4, padx = 15)
            self.label_2.grid(row = 1, column = 5, columnspan = 2, sticky = "w")
            
class ReportingFrame(tk.Frame):
    def __init__(self, root, pairings, previous_scores = None, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        self.master = root
        
        self.rowconfigure(0, weight = 1)
        self.columnconfigure([1,5], minsize = 100)
        
        self.index = 0
        # We need a list of pairings 
        self.pairings = pairings
        self.report_slips = []
        self.previous_scores = previous_scores
        
        # Create the pairings
        self.present_pairings(pairings)
        
        # Create frame for buttons
        self.buttonframe = ButtonFrame(self, self.pairings)
        self.buttonframe.grid(row = self.index + 1, columnspan = 5, sticky = "nsew")
        
    def present_pairings(self, pairings):
        """
        Creates the results_frames for the given pairings
        """
        for pair_num, pair in enumerate(pairings):
            # Used to put the rows nicely onto the screen
            self.index += 1
            if "Bye" in pair or "bye" in pair:
                self.resultsframe = ResultFrame(self, pair = pair, is_bye = True)
                self.resultsframe.grid(row = self.index, 
                                       column = 1, 
                                       columnspan = 5,
                                       sticky = "nsew")
            else:
                if self.previous_scores is not None:
                    self.resultsframe = ResultFrame(self, 
                                                    pair = pair, 
                                                    is_bye = False, 
                                                    score = self.previous_scores[pair_num])
                else:
                    self.resultsframe = ResultFrame(self, pair = pair, is_bye = False)
                self.resultsframe.grid(row = self.index, 
                                       column = 1, 
                                       columnspan = 5, 
                                       sticky = "nsew")
            
            # Append these frame instances to report_slips - we will use these to get
            # the reported values later on
            self.report_slips.append(self.resultsframe)
    
    def undo_results(self):
        pass
        
class StandingsFrame(tk.Frame):
    def __init__(self, root, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        self.master = root
        
        self.columnconfigure(1, weight = 1)
        self.rowconfigure(1, weight = 1)
        
        self.tree = ttk.Treeview(self, show = 'headings')
        self.tree.grid(row = 1, column = 1, sticky = "nsew")
        
        # Add the columns to the tree.columns attribute. <- this needs to be an iterable.
        # Remember to update these with tournamentorganizer.standings_df
        self.tree["columns"] = list(self.master.tournamentorganizer.standings_df.columns)
        
        for header in list(self.master.tournamentorganizer.standings_df.columns):
            # This sets up the column itself
            # tree.column basically does things to the column name specified in tree["columns"]
            self.tree.column(header, width = 100, anchor = "w")
            # This sets up the header bar
            self.tree.heading(header, 
                              text = header, 
                              anchor = "w", 
                              command = lambda _col = header: self.treeview_sort_column(self.tree, 
                                                                                        _col, 
                                                                                        False))
        for index, row in self.master.tournamentorganizer.standings_df.iterrows():
            # Note that the second argument is where the thing is inserted. 0 inserts it at the
            # top. In other words, the last entry of the list will be at the top
            # tree.insert("", 0, text = index, values = list(row))

            # Using "end" inserts the thingy at the end instead.
            self.tree.insert("", "end", text = index, values = list(row))
    
    def treeview_sort_column(self, tv, col, reverse):
        """
        Allows the columns to be sorted in reverse
        """
        l = [(tv.set(k, col), k) for k in tv.get_children('')]
        l.sort(reverse = reverse)

        # rearrange items in sorted positions
        for index, (val, k) in enumerate(l):
            tv.move(k, '', index)

        # reverse sort next time
        tv.heading(col, 
                   text=col, 
                   command = lambda _col = col: self.treeview_sort_column(tv,
                                                                          _col,
                                                                          not reverse))

# The frame that comes at the end of the tourney
class EndFrame(tk.Frame):
    def __init__(self, root, *args, **kwargs):
        super().__init__(root, *args, **kwargs)
        self.master = root
        
        # Some dummy frames
        self.info_frame = tk.Frame()
        self.extras_frame = tk.Frame()
        
        self.columnconfigure(0, weight = 1)
        self.rowconfigure([0,1], weight = 1)
        
        self.setup_end_screen()
        
    def setup_end_screen(self):
        # Frame containing some blurb and ability to save map history
        self.info_frame = tk.Frame(self)
        self.info_frame.grid(row = 0, column = 0, pady = 30, sticky = "nsew")
        self.info_frame.columnconfigure([0,1,2], minsize = 100)
        
        info_label = tk.Label(self.info_frame, text = "The Tournament has ended.\n\nThe final standings are presented on the right.\nYou can save the match history using\nthe button on the right.\n\nThanks for playing!")
        info_label.grid(row = 1, column = 0, sticky = "nsew", padx = 50)
        
        # Save Match History
        save_history_button = tk.Button(self.info_frame, 
                                      text = "Save Current Match History", 
                                      # Save history function located in the Main App class
                                      command = self.master.save_history)
        save_history_button.grid(row = 1, column = 1, columnspan = 2, pady = 20, padx = 25)
        
        # Add extras to the End Screen
        self.extras_frame = tk.Frame(self)
        self.extras_frame.grid(row = 1, column = 0, pady = 30, sticky = "nsew")
        self.extras_frame.columnconfigure([0,1,2], minsize = 100)
        
        elo_label = tk.Label(self.extras_frame, text = "Track the ELO of your players!\n\n"
                             "Create a new ELO tracking file or load a CSV file\ncontaining"
                             " pre-existing ELO records. Must contain\na 'Player' and 'ELO' column."
                             "\n\nNote that loading an ELO file will ask you to\nload and save a file"
                             "in quick succession.").grid(row = 0, column = 0, pady = 50, padx = 20)
        
        create_elo_button = tk.Button(self.extras_frame, 
                                      text = "Create ELO file", 
                                      command = self.handle_elo_csv)
        load_elo_button = tk.Button(self.extras_frame, 
                                    text = "Update ELO file", 
                                    command = lambda: self.handle_elo_csv(update = True))
        
        create_elo_button.grid(row = 0, column = 1, pady = 10, padx = 20, sticky = "ew")
        load_elo_button.grid(row = 0, column = 2, pady = 10, padx = 20, sticky = "ew")
    
    def expected_score(self, player_rating, opponent_rating):
        """
        What is a player's expected score considering their own vs. their opponent's rating.
        """
        QA = 10**(player_rating / 400)
        QB = 10**(opponent_rating / 400)
        return QA/(QA+QB)

    def update_elo(self, player_rating, expected_score, actual_score, k_factor = 20):
        """
        Recalculates ELO given the results of a tournament.

        Actual/Expected scores must be the sum of the results from the tournament
        """
        new_rating = player_rating + k_factor * (actual_score - expected_score)
        return new_rating
    
    def handle_elo_csv(self, update = False):
        """
        Loads and makes changes to the elo_csv.
        """
        
        if update:
            filepath = askopenfilename(
                filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
            )

            # If no file selected, do nothing
            if not filepath:
                return

            # Load initial elo
            current_elo_df = pd.read_csv(filepath)

            # Dictionary of players initial ELOs
            player_initial_elo = dict(zip(current_elo_df.Player, current_elo_df.ELO))

            # Find new players
            new_players = []
            
            for p in self.master.tournamentorganizer.player_list:
                if p not in player_initial_elo.keys():
                    new_players.append(p)
            
            # new_players = list(set(self.master.tournamentorganizer.player_list).symmetric_difference(set(player_initial_elo.keys())))

            # If there are new players, initialize them at 1500 ELO
            for player in new_players:
                player_initial_elo[player] = 1500
        
        else:
            # If new ELO creator
            player_initial_elo = {name:1500 for name in self.master.tournamentorganizer.player_list}
        
        # Create a dictionary to store expected scores
        player_expected_scores = {name: [] for name in player_initial_elo.keys()}

        # Go through each match to get expected scores
        for match in self.master.tournamentorganizer.tournament_history:
            # If there is a bye, ignore it completely
            if self.master.tournamentorganizer.tournament_history[match]["Player_2"] == "Bye":
                continue

            player_one = self.master.tournamentorganizer.tournament_history[match]["Player_1"]
            player_two = self.master.tournamentorganizer.tournament_history[match]["Player_2"]

            # Calculate expected scores
            player_expected_scores[player_one].append(self.expected_score(player_initial_elo[player_one],
                                                                     player_initial_elo[player_two]))
            player_expected_scores[player_two].append(self.expected_score(player_initial_elo[player_two],
                                                                     player_initial_elo[player_one]))

        # Create a dictionary containing each player's new ELO
        # Round each player's ELO to a whole number.
        player_new_elo = dict()
        
        
        
        for name in player_initial_elo:
            try:
                new_elo = np.round(self.update_elo(player_initial_elo[name], 
                                       sum(player_expected_scores[name]), 
                                       self.master.tournamentorganizer.standings_df.loc[self.master.tournamentorganizer.standings_df["Player"] == name,"Points"].values[0] - \
                                                   self.master.tournamentorganizer.has_bye(name), 
                                       k_factor = 20))
                player_new_elo[name] = new_elo
            except:
                player_new_elo[name] = player_initial_elo[name]
        
        # Does not work for names that are not in the tournament
#         player_new_elo = {name: np.round(self.update_elo(player_initial_elo[name], 
#                                        sum(player_expected_scores[name]), 
#                                        self.master.tournamentorganizer.standings_df.loc[self.master.tournamentorganizer.standings_df["Player"] == name,"Points"].values[0], 
#                                        k_factor = 20)) for name in player_initial_elo}

        # Convert to dataframe
        new_elo_df = pd.DataFrame.from_dict(player_new_elo, orient = "index").reset_index()
        new_elo_df.columns = ["Player", "ELO"]
        new_elo_df.sort_values("ELO", ascending = False, inplace = True)
        
        # Save the elo to a specific place
        filepath = asksaveasfilename(
            defaultextension="csv",
            filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
        )

        # If no file selected, do nothing
        if not filepath:
            return
        
        # Save to a new file containing today's date
        new_elo_df.to_csv(filepath)
        
        

        
# The app that runs the actual tournament. 
class MainApplication(tk.Tk):
    def __init__(self, tournamentorganizer, previous_pairings = None, previous_scores = None):
        super().__init__()
        # The tournament organizer class necessary to run all the functions
        self.tournamentorganizer = tournamentorganizer
        # configure the root window
        self.title('Swiss Tournament Manager')
        self.rowconfigure(1, minsize = 300, weight = 1)
        self.columnconfigure(2, minsize = 400, weight = 1)
        
        # For any previous pairings
        self.previous_pairings = previous_pairings
        print(self.previous_pairings)
        self.previous_scores = previous_scores
        
        # Placeholder frames
        self.reportingframe = tk.Frame()
        self.standingsframe = tk.Frame()
        self.refresh_window()
        
        self.errorframe = ErrorFrame(self)
    
    def refresh_window(self):
        """
        Refreshes the window by finding the destroying the old window and making new
        frames
        """
        self.reportingframe.destroy()
        # Pairings are created here. If there are pairings from a loaded match_history
        # we want to use them instead.
        if self.previous_pairings is not None:
            self.reportingframe = ReportingFrame(self, 
                                                 pairings = self.previous_pairings, 
                                                 previous_scores = self.previous_scores)
            # Then remove the previous pairings - we don't want them around
            self.previous_pairings = None
        else:
            # If there are no previous pairings, we create our own
            self.reportingframe = ReportingFrame(self, 
                                                 pairings = self.create_pairings(), 
                                                 previous_scores = self.previous_scores)
        self.reportingframe.grid(row = 1, column = 1, sticky = "nsew")
        
        # Purge previous scores
        self.previous_scores = None
        self.standingsframe.destroy()
        self.standingsframe = StandingsFrame(self, relief = tk.SUNKEN, borderwidth = 3)
        self.standingsframe.grid(row = 1, column = 2, sticky = "nsew")
    
    def create_pairings(self):
        return self.tournamentorganizer.determine_pairings()
    
    def submit_results(self):
        # Increment round number
        self.tournamentorganizer.round_num += 1
        
        # Check if report slip has been filled in correctly.
        for reportframe in self.reportingframe.report_slips:
            # Get the results from the spinboxes - raise error if non-number
            try:
                if reportframe.is_bye is False:
                    float(reportframe.score_1.get())
                    float(reportframe.score_2.get())
            except:
                self.raise_error(f"Error: Match {self.reportingframe.report_slips.index(reportframe) + 1} has a non-number score.")
                return
        
        # report_slips is a list of ResultFrames containing the reported results
        for reportframe in self.reportingframe.report_slips:
            # If there was a "Bye", report 1-0
            if reportframe.is_bye:
                # "Bye" should always be the second of the two pairings
                self.tournamentorganizer.add_result(player_one = reportframe.pair[0], 
                                                    player_two = "Bye", 
                                                    player_one_score = 1, 
                                                    player_two_score = 0)
            
            # If not bye, extract information and add to the tournamentorganizer
            else:
                self.tournamentorganizer.add_result(reportframe.pair[0], 
                                                    reportframe.pair[1], 
                                                    float(reportframe.score_1.get()), 
                                                    float(reportframe.score_2.get()))
        
        # Update the standings
        self.tournamentorganizer.update_standings()
        
        # If the tourney has ended, create an end of tournament screen
        if self.reportingframe.buttonframe.end_tourney.get() == 1:
            self.end_of_tourney_screen()
        # Recreate the reporting/standings frames with the new pairings
        else:
            self.refresh_window()
        
    def drop_player(self):
        """
        Drops the player name in the "drop_entry" Entry.

        Weird quirk. Current behaviour is that you have to re-drop all dropped players if you
        undo a result.

        Generates errors as necessary.
        """
        self.close_error()
        dropped_player = self.reportingframe.buttonframe.drop_entry.get()

        # print(dropped_player)

        # Attempt to remove the player from the list of players
        if dropped_player in self.tournamentorganizer.player_list:
            # Drop player
            self.tournamentorganizer.player_list.pop(self.tournamentorganizer.player_list.index(dropped_player))

            # Print confirmation message.
            self.raise_error(f"{dropped_player} successfully dropped.")
            return

        else:
            # Inform user that player was not found.
            self.raise_error(f"Error: {dropped_player} not found in tournament. Please check spelling.")
            return
        
    def save_history(self):
        """
        Saves the current match history. Does NOT save pairings.
        """
        # Opens a dialogue that asks to save the file
        filepath = asksaveasfilename(
            defaultextension="csv",
            filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")],
        )

        # If nothing was entered, do nothing
        if not filepath:
            return
        
        # Create match history df
        match_history_df = pd.DataFrame.from_dict(self.tournamentorganizer.tournament_history).transpose()
        
#         # If the round is incomplete
#         if self.reportingframe.buttonframe.round_incomplete:
            # Note that this saves a bit weirdly but its correct
        
        # Save the current pairings and scores
        with open(filepath, 'w') as f:
            f.write('# Round incomplete\n')
            f.write(f'#player_list{self.tournamentorganizer.player_list}\n')
            # The pairings
            for report in self.reportingframe.report_slips:
                if report.is_bye:
                    f.write(f'# pairing,{report.pair[0]},bye\n')
                else:
                    # Note that if there is no value in the report frame, then it will be saved as ""
                    f.write(f'# pairing,{report.pair[0]},{report.pair[1]},{report.score_1.get()},{report.score_2.get()}\n')
            match_history_df.to_csv(f)
        
#         # If round was complete, no other things needed
#         else:
#             match_history_df.to_csv(filepath)
    
    def undo_results(self):
        self.tournamentorganizer.undo_results()
        self.tournamentorganizer.update_standings()
        
        # Recreate the reporting/standings frames with the new pairings
        self.refresh_window()
    
    def raise_error(self, error_message):
        self.errorframe = ErrorFrame(self, error_message)
        self.errorframe.grid(row = 4, column = 1, pady = 20)
        
    def close_error(self):
        if self.errorframe.winfo_exists():
            self.errorframe.destroy()
            
    def end_of_tourney_screen(self):
        # Replace the reporting frame with a different thing
        self.reportingframe.destroy()
        self.reportingframe = EndFrame(self, borderwidth = 3)
        self.reportingframe.grid(row = 1, column = 1, sticky = "nsew")
        
        
        # Create the standings frame as per usual
        self.standingsframe.destroy()
        self.standingsframe = StandingsFrame(self, relief = tk.SUNKEN, borderwidth = 3)
        self.standingsframe.grid(row = 1, column = 2, sticky = "nsew")
    

In [4]:
first_app = SetupApplication()
first_app.mainloop()

None


In [5]:
# with open("incomplete testing.csv", "r") as f:
#     # Note that once you call readline, the "cursor" moves to the next line already
#     if 'incomplete' in f.readline():
#         print("Need to load new data")
#     for line in f.readlines():
#         if 'player_list' in line:
#             test = [player.strip("'") for player in line[line.find("[")+1:line.find("]")].split(", ")]
            
# test